## Kaggle – DataTops®
Tu profe ha decidido cambiar de aires y, por eso, ha comprado una tienda de portátiles. Sin embargo, su única especialidad es Data Science, por lo que ha decidido crear un modelo de ML para establecer los mejores precios.

¿Podrías ayudar a tu profe a mejorar ese modelo?

## Métrica: 
Error de raíz cuadrada media (RMSE) es la desviación estándar de los valores residuales (errores de predicción). Los valores residuales son una medida de la distancia de los puntos de datos de la línea de regresión; RMSE es una medida de cuál es el nivel de dispersión de estos valores residuales. En otras palabras, le indica el nivel de concentración de los datos en la línea de mejor ajuste.


$$ RMSE = \sqrt{\frac{1}{n}\Sigma_{i=1}^{n}{\Big(\frac{d_i -f_i}{\sigma_i}\Big)^2}}$$


## Librerías

In [1]:
import numpy as np
import pandas as pd
import toolbox_ML as tb
import re
from PIL import Image
from sklearn.model_selection import train_test_split, GridSearchCV
from xgboost import XGBRegressor
import urllib.request

## Datos

In [2]:
# Carga de datos
df = pd.read_csv("./data/train.csv", index_col="laptop_ID")
test = pd.read_csv("./data/test.csv", index_col="laptop_ID")

# Convertimos nombres de columnas a minúsculas
df.columns = df.columns.str.lower()
test.columns = test.columns.str.lower()

## Exploración de los datos

In [3]:
display(df)
print(df.info())

,company,product,typename,inches,screenresolution,cpu,ram,memory,gpu,opsys,weight,price_in_euros
laptop_ID,,,,,,,,,,,,
755,HP,250 G6,Notebook,15.6,Full HD 1920x1080,Intel Core i3 6006U 2GHz,8GB,256GB SSD,Intel HD Graphics 520,Windows 10,1.86kg,539.00
618,Dell,Inspiron 7559,Gaming,15.6,Full HD 1920x1080,Intel Core i7 6700HQ 2.6GHz,16GB,1TB HDD,Nvidia GeForce GTX 960<U+039C>,Windows 10,2.59kg,879.01
909,HP,ProBook 450,Notebook,15.6,Full HD 1920x1080,Intel Core i7 7500U 2.7GHz,8GB,1TB HDD,Nvidia GeForce 930MX,Windows 10,2.04kg,900.00
2,Apple,Macbook Air,Ultrabook,13.3,1440x900,Intel Core i5 1.8GHz,8GB,128GB Flash Storage,Intel HD Graphics 6000,macOS,1.34kg,898.94
286,Dell,Inspiron 3567,Notebook,15.6,Full HD 1920x1080,Intel Core i3 6006U 2.0GHz,4GB,1TB HDD,AMD Radeon R5 M430,Linux,2.25kg,428.00
...,...,...,...,...,...,...,...,...,...,...,...,...
28,Dell,Inspiron 5570,Notebook,15.6,Full HD 1920x1080,Intel Core i5 8250U 1.6GHz,8GB,256GB SSD,AMD Radeon 530,Windows 10,2.2kg,800.00
1160,HP,Spectre Pro,2 in 1 Convertible,13.3,Full HD / Touchscreen 1920x1080,Intel Core i5 6300U 2.4GHz,8GB,256GB SSD,Intel HD Graphics 520,Windows 10,1.48kg,1629.00
78,Lenovo,IdeaPad 320-15IKBN,Notebook,15.6,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8GB,2TB HDD,Intel HD Graphics 620,No OS,2.2kg,519.00


<class 'pandas.core.frame.DataFrame'>
Index: 912 entries, 755 to 229
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   company           912 non-null    object 
 1   product           912 non-null    object 
 2   typename          912 non-null    object 
 3   inches            912 non-null    float64
 4   screenresolution  912 non-null    object 
 5   cpu               912 non-null    object 
 6   ram               912 non-null    object 
 7   memory            912 non-null    object 
 8   gpu               912 non-null    object 
 9   opsys             912 non-null    object 
 10  weight            912 non-null    object 
 11  price_in_euros    912 non-null    float64
dtypes: float64(2), object(10)
memory usage: 92.6+ KB
None


In [4]:
tb.describe_df(df)

,company,product,typename,inches,screenresolution,cpu,ram,memory,gpu,opsys,weight,price_in_euros
DATA_TYPE,object,object,object,float64,object,object,object,object,object,object,object,float64
MISSINGS (%),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
UNIQUE_VALUES,19,480,6,17,36,107,9,37,93,9,165,603
CARDIN (%),2.08,52.63,0.66,1.86,3.95,11.73,0.99,4.06,10.2,0.99,18.09,66.12


## Procesado de datos

En primer lugar añadimos una celda de código necesaria pues, en una primera ejecución de XGBoost, teníamos problemas con los caracteres ">" y "<" incluidos en "Nvidia GeForce GTX 960<U+039C>", una de las gráficas que pasa a ser una columna al hacer el one hot encoding. Ese código entre esos dos símbolos hace referencia a la letra $\mu$, que al tabular los datos queda reflejado con su codificación en unicode.

In [5]:
# Sustituimos los símbolos que luego nos daban problemas en XGBoost
df["gpu"] = df["gpu"].str.replace("<", "", regex=False).str.replace(">", "", regex=False)
test["gpu"] = test["gpu"].str.replace("<", "", regex=False).str.replace(">", "", regex=False)

Después de hacer un value counts, convertimos a numéricas de un primer plumazo las tres columnnas que son más inmediatas. La ram, expresada en GB, el peso, en kg, y la resolución vertical y horizontal que extraemos de la columna screen resolution.

In [6]:
print(df.ram.value_counts())
print(df.weight.value_counts())
print(df.screenresolution.value_counts())

ram
8GB     434
4GB     267
16GB    136
6GB      24
2GB      20
12GB     19
32GB     10
64GB      1
24GB      1
Name: count, dtype: int64
weight
2.2kg     91
2.1kg     40
2.4kg     31
2.5kg     29
2.3kg     27
          ..
0.91kg     1
2.15kg     1
2.54kg     1
1.18kg     1
4.33kg     1
Name: count, Length: 165, dtype: int64
screenresolution
Full HD 1920x1080                                349
1366x768                                         211
IPS Panel Full HD 1920x1080                      163
IPS Panel Full HD / Touchscreen 1920x1080         32
Full HD / Touchscreen 1920x1080                   30
1600x900                                          14
Quad HD+ / Touchscreen 3200x1800                  11
Touchscreen 1366x768                              11
IPS Panel 4K Ultra HD / Touchscreen 3840x2160     10
4K Ultra HD / Touchscreen 3840x2160                7
Touchscreen 2560x1440                              6
IPS Panel Quad HD+ / Touchscreen 3200x1800         6
IPS Panel 4K Ultra H

In [7]:
# Convertimos algunas a numéricas
df["ram_in_gb"] = df["ram"].str.replace("GB", "").astype(float)
test["ram_in_gb"] = test["ram"].str.replace("GB", "").astype(float)

df["weight_in_kg"] = df["weight"].str.replace("kg", "").astype(float)
test["weight_in_kg"] = test["weight"].str.replace("kg", "").astype(float)

df["res_x"] = df["screenresolution"].str.extract(r"(\d+)[xX]", expand=False).astype(float)
df["res_y"] = df["screenresolution"].str.extract(r"[xX](\d+)", expand=False).astype(float)

test["res_x"] = test["screenresolution"].str.extract(r"(\d+)[xX]", expand=False).astype(float)
test["res_y"] = test["screenresolution"].str.extract(r"[xX](\d+)", expand=False).astype(float)

Ahora usaremos los rawstrings y las conversiones entre GB y TB para separar la columna de almacenamiento en dos numéricas que recojan toda la información. Por un lado la memoria SSD y por otro lado la total (para tener en cuenta también los que tengan discos duros o demás almacenamiento externo).

In [8]:
print(df.memory.value_counts())

memory
256GB SSD                        282
1TB HDD                          152
500GB HDD                         92
512GB SSD                         83
128GB SSD +  1TB HDD              67
128GB SSD                         54
256GB SSD +  1TB HDD              52
32GB Flash Storage                33
1TB SSD                           12
64GB Flash Storage                11
2TB HDD                            8
512GB SSD +  1TB HDD               8
256GB Flash Storage                7
256GB SSD +  2TB HDD               6
16GB Flash Storage                 6
1.0TB Hybrid                       5
32GB SSD                           5
128GB Flash Storage                4
180GB SSD                          3
16GB SSD                           3
1TB SSD +  1TB HDD                 2
512GB SSD +  2TB HDD               2
256GB SSD +  256GB SSD             1
128GB SSD +  2TB HDD               1
512GB SSD +  512GB SSD             1
64GB Flash Storage +  1TB HDD      1
64GB SSD                       

In [9]:
def parse_storage(text):
    # Expresiones para detectar cada tipo y cantidad
    total = 0
    ssd = 0

    for part in text.split("+"): #divido en dos partes si hay almacenamiento combinado
        part = part.strip().upper() #analizo cada parte
        if "SSD" in part or "FLASH" in part: #filtramos si es memoria SSD o Flash Storage y en ese caso guardaré en ssd y en total
            if "TB" in part:
                value = float(re.search(r"(\d+\.?\d*)TB", part).group(1)) * 1024 #convertimos a GB
            elif "GB" in part:
                value = float(re.search(r"(\d+\.?\d*)GB", part).group(1))
            else:
                value = 0
            ssd += value
            total += value
        #aquí ya estaríamos dentro de los casos HDD, filtramos otra vez entre TB y GB para hacer la conversión
        elif "TB" in part: 
            total += float(re.search(r"(\d+\.?\d*)TB", part).group(1)) * 1024
        elif 'GB' in part:
            total += float(re.search(r"(\d+\.?\d*)GB", part).group(1))
    
    return pd.Series([total, ssd])

# Aplicamos a train y test
df[["total_storage_gb", "ssd_storage_gb"]] = df["memory"].apply(parse_storage)
test[["total_storage_gb", "ssd_storage_gb"]] = test["memory"].apply(parse_storage)

Vemos en un info las columnas que nos han quedado y sus tipos:

In [10]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Index: 912 entries, 755 to 229
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   company           912 non-null    object 
 1   product           912 non-null    object 
 2   typename          912 non-null    object 
 3   inches            912 non-null    float64
 4   screenresolution  912 non-null    object 
 5   cpu               912 non-null    object 
 6   ram               912 non-null    object 
 7   memory            912 non-null    object 
 8   gpu               912 non-null    object 
 9   opsys             912 non-null    object 
 10  weight            912 non-null    object 
 11  price_in_euros    912 non-null    float64
 12  ram_in_gb         912 non-null    float64
 13  weight_in_kg      912 non-null    float64
 14  res_x             912 non-null    float64
 15  res_y             912 non-null    float64
 16  total_storage_gb  912 non-null    float64
 17  

In [11]:
# Ahora hacemos un one-hot encoding de las categóricas
df = pd.get_dummies(df, columns=["company", "product", "typename", "cpu", "gpu", "opsys"], drop_first=True)
test = pd.get_dummies(test, columns=["company", "product", "typename", "cpu", "gpu", "opsys"], drop_first=True)

---

## Modelado

### 1. Definir X e y

In [12]:
# Selección de features
X = df.drop(columns=["price_in_euros", "screenresolution", "ram", "memory", "weight"])
y = df["price_in_euros"]

### 2. Dividir X_train, X_test, y_train, y_test

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 42)

### 3. Modelo

Después de haber hecho GridSearch con el split train-test los mejores hiperparámetros eran estos. Así que para no perder información entrenamos ahora con TODO el dataset para hacer luego el submit.

In [14]:
best_model = XGBRegressor(
    objective='reg:squarederror',
    random_state=42,
    n_estimators=740,
    learning_rate=0.0635,
    max_depth=7,
    colsample_bytree=0.36,
    subsample=0.46
)

best_model.fit(X, y)

# Predicción para el conjunto de test de Kaggle
X_kaggle_test = test.reindex(columns=X.columns, fill_value=0)
predictions_submit = best_model.predict(X_kaggle_test)

## Kaggle submission

Visualizamos el sample submission de ejemplo para copiar su estructura:

In [15]:
sample = pd.read_csv("./data/sample_submission.csv")
display(sample.head())
print(sample.shape)

,laptop_ID,Price_in_euros
0,209,1949.1
1,1281,805.0
2,1168,1101.0
3,1231,1293.8
4,1020,1832.6


(391, 2)


Creamos lo que será nuestra submission basándonos en la de ejemplo:

In [16]:
# Creamos el DataFrame de submission con la estructura correcta
submission = pd.DataFrame({
    "laptop_ID": test.index,
    "Price_in_euros": predictions_submit
})
display(submission.head())
print(submission.shape)

,laptop_ID,Price_in_euros
0,209,1284.837524
1,1281,315.118347
2,1168,368.480835
3,1231,970.736023
4,1020,1024.315430


(391, 2)


Comprobamos con el chequeador que está todo correcto y generamos el csv que subiremos:

In [17]:
def chequeador(df_to_submit):
    """
    Esta función se asegura de que tu submission tenga la forma requerida por Kaggle.
    
    Si es así, se guardará el dataframe en un `csv` y estará listo para subir a Kaggle.
    
    Si no, LEE EL MENSAJE Y HAZLE CASO.
    
    Si aún no:
    - apaga tu ordenador, 
    - date una vuelta, 
    - enciendelo otra vez, 
    - abre este notebook y 
    - leelo todo de nuevo. 
    Todos nos merecemos una segunda oportunidad. También tú.
    """
    if df_to_submit.shape == sample.shape:
        if df_to_submit.columns.all() == sample.columns.all():
            if df_to_submit.laptop_ID.all() == sample.laptop_ID.all():
                print("You're ready to submit!")
                submission.to_csv("submission_xgb.csv", index = False) #muy importante el index = False
                urllib.request.urlretrieve("https://www.mihaileric.com/static/evaluation-meme-e0a350f278a36346e6d46b139b1d0da0-ed51e.jpg", "gfg.png")     
                img = Image.open("gfg.png")
                img.show()   
            else:
                print("Check the ids and try again")
        else:
            print("Check the names of the columns and try again")
    else:
        print("Check the number of rows and/or columns and try again")
        print("\nMensaje secreto del TA: No me puedo creer que después de todo este notebook hayas hecho algún cambio en las filas de `test.csv`. Lloro.")

In [18]:
chequeador(submission)

You're ready to submit!
